In [19]:
import networkx as nx
import pandas as pd
import numpy as np
import csv

In [20]:
data = pd.read_csv("./project_hpa_protein_data/lung.tsv", delimiter = '\t', header = None, usecols = [0,1])
data

,0,1
0,ENSG00000105483,ENSG00000171467
1,ENSG00000100030,ENSG00000261371
2,ENSG00000162385,ENSG00000169045
3,ENSG00000141458,ENSG00000184584
4,ENSG00000117724,ENSG00000197321
...,...,...
102367,ENSG00000138131,ENSG00000138190
102368,ENSG00000107581,ENSG00000135480
102369,ENSG00000094631,ENSG00000239713
102370,ENSG00000167900,ENSG00000196705


In [21]:
krogan_proteins = pd.read_excel("./krogan_proteins.xlsx", usecols= [0,1])
krogan_proteins

,Gene stable ID,Gene name
0,ENSG00000280759,AP2A2
1,ENSG00000228691,NEU1
2,ENSG00000282003,TUBGCP3
3,ENSG00000224774,CSNK2B
4,ENSG00000282230,ADAM9
...,...,...
327,ENSG00000198933,TBKBP1
328,ENSG00000136485,DCAF7
329,ENSG00000154240,CEP112
330,ENSG00000109062,SLC9A3R1


In [22]:
huMAP1 = pd.read_csv("./humap2_complexes_20200809.txt", delimiter = ',')

In [23]:
huMAP = pd.Series(huMAP1['genenames'])
COI = []
kg = list(krogan_proteins['Gene name'])
for i in huMAP:
    if any(item in kg for item in i.split(' ')):
        COI.append(i.split(' '))

file = open('complexes_of_interest.csv', 'w+', newline ='') 
  
# writing the data into the file 
with file:     
    write = csv.writer(file) 
    write.writerows(COI) 

file.close()

In [33]:
from scipy.stats import fisher_exact

gba = pd.read_csv("./gba_gene_names.txt", delimiter = '\t')
gba = list(gba['Gene Symbol'])

diff = pd.read_csv("./diff_gene_names.txt", delimiter = '\t', header = None)
diff = list(diff[1])

rwr = pd.read_csv("./RWR_gene_names.txt", delimiter = '\t', header = None)
rwr = list(rwr[1])
gba


['NUP214',
 'NUP62',
 'RANBP2',
 'NUP93',
 'VCP',
 'OBSL1',
 'UBQLN1',
 'HTT',
 'NUP107',
 'APC',
 'HSPA9',
 'SMURF1',
 'HSPA5',
 'KIF5B',
 'SEC16A',
 'UBC',
 'AIFM1',
 'WWOX',
 'KPNB1',
 'ITCH',
 'CLTC',
 'WWP2',
 'NCL',
 'TUBA1A',
 'MED20',
 'EXOSC3',
 'SNRPB',
 'EXOSC10',
 'MTREX',
 'DIS3',
 'RBM7',
 'TMPO',
 'REL',
 'MDC1',
 'RPA3',
 'EXOSC8',
 'EXOSC2',
 'TP53',
 'XRN1',
 'MPHOSPH6',
 'RPA1',
 'ATF2',
 'EXOSC5',
 'MED4',
 'EXOSC4',
 'RPA2',
 'DIS3L',
 'EXOSC9',
 'NXF1',
 'EXOSC7',
 'SNRPC',
 'EXOSC1',
 'XPO1',
 'HECW2',
 'BAG6',
 'ELAVL1',
 'PTEN',
 'LYN',
 'NEDD4',
 'TUBGCP3',
 'OFD1',
 'GAN',
 'CALM1',
 'YWHAZ',
 'CCDC8',
 'CDC5L',
 'TSNAX',
 'CDK5RAP2',
 'MAPRE1',
 'DISC1',
 'CUL3',
 'CUL2',
 'TRAF6',
 'MAP3K7',
 'KRT40',
 'MCC',
 'FBXO6',
 'RNF2',
 'FN1',
 'SIRT4',
 'CCDC85B',
 'HERC2',
 'TUBG1',
 'POU5F1',
 'HAUS1',
 'EZH2',
 'KAT5',
 'RCOR3',
 'BRCA1',
 'KANSL1',
 'ENO1',
 'H2AX',
 'DERL1',
 'UFD1',
 'SRPRB',
 'SYVN1',
 'DDX19B',
 'VDAC2',
 'SEL1L',
 'EMD',
 'RPN1',
 'NDUFAF

## Calculating Fisher's Exact Test

In [34]:
# for each complex finding var1,var2,var3,var4 which are the 4 variables in the contingency matrix

#guilt-by-association
total_gba = 2930
gba_th4 = len(gba)
not_gba_th4 = total_gba - len(gba)

file = open('enriched_complexes_gba.csv', 'w+', newline ='')

file.write("complex,odds ratio, p-value\n")
for comp in COI:
    len_comp = len(comp)
    var1 = len(set(comp).intersection(gba) )
    var2 = len_comp - var1
    var3 = gba_th4 - var1
    var4 = not_gba_th4 - var2
    oddsratio, pvalue = fisher_exact([[var1,var2],[var3,var4]])
    if pvalue < 0.05:
        #print(var1,var2,var3,var4)
        #print(pvalue)
        c = " ".join(comp)
        file.write(c+","+str(oddsratio)+","+str(pvalue)+"\n")
file.close()

#network diffusion
total_ppi = 9095
diff_th005 = len(diff)
not_diff_th005 = total_ppi - len(diff)

file = open('enriched_complexes_diff.csv', 'w+', newline ='')

file.write("complex,odds ratio, p-value\n")
for comp in COI:
    len_comp = len(comp)
    var1 = len(set(comp).intersection(diff) )
    var2 = len_comp - var1
    var3 = diff_th005 - var1
    var4 = not_diff_th005 - var2
    oddsratio, pvalue = fisher_exact([[var1,var2],[var3,var4]])
    if pvalue < 0.05:
        #print(var1,var2,var3,var4)
        #print(pvalue)
        c = " ".join(comp)
        file.write(c+","+str(oddsratio)+","+str(pvalue)+"\n")
file.close()

#random walks with restart
total = 9095
rwr_th0005 = len(rwr)
not_rwr_th0005 = total - len(rwr)

file = open('enriched_complexes_rwr.csv', 'w+', newline ='')

file.write("complex,odds ratio, p-value\n")
for comp in COI:
    len_comp = len(comp)
    var1 = len(set(comp).intersection(rwr) )
    var2 = len_comp - var1
    var3 = rwr_th0005 - var1
    var4 = not_rwr_th0005 - var2
    oddsratio, pvalue = fisher_exact([[var1,var2],[var3,var4]])
    if pvalue < 0.05:
        #print(var1,var2,var3,var4)
        #print(pvalue)
        c = " ".join(comp)
        file.write(c+","+str(oddsratio)+","+str(pvalue)+"\n")
file.close()

### Finding complexes common to all three methods

In [17]:
gba = pd.read_csv("./enriched_complexes_gba.csv")
diff = pd.read_csv("./enriched_complexes_diff.csv")
rwr = pd.read_csv("./enriched_complexes_rwr.csv")

common = list(set(gba['complex']) & set(diff['complex']) & set(rwr['complex']))
common

['AKAP9 PRKACG AKAP1 PRKAR2B PRKAR2A PRKACA PRKACB AKAP11 AKAP14 CDK5RAP2 AKAP5',
 'EXOSC3 EXOSC5',
 'EXOSC9 RBM7 EXOSC6 EXOSC10 EXOSC3 DIS3L EXOSC1 EXOSC4 C1D EXOSC2 EXOSC8 EXOSC5 MPHOSPH6 EXOSC7 ZCCHC8',
 'NUP62 NUP88',
 'NUP214 NUP88',
 'NUP54 NUP214 NUP88 NUP62',
 'NUP62 NUP58 NUP188 EIF4G3 NUP88 NUP214',
 'ERLEC1 SEL1L OS9',
 'RAB5C RAB5A RAB7A',
 'NUP214 NUP88 NUP62',
 'CHM RAB5B RAB5A RAB5C RAP1B RAB7A RAB2A',
 'EXOSC9 RBM7 EXOSC6 EXOSC10 EXOSC3 DIS3L EXOSC1 EXOSC4 C1D EXOSC2 EXOSC8 EXOSC5 MPHOSPH6 EXOSC7']